# GRUPO 3 - Great Expectations

Luz Alizon

Mauricio

Victor Quino

Great Expectations (GE) es una herramienta de código abierto diseñada para asegurar la calidad de los datos en proyectos de análisis y ciencia de datos. Permite definir “expectativas” o reglas sobre el dataset (por ejemplo: que una columna no tenga valores nulos, que los precios sean positivos, que las fechas estén en un rango válido) y luego verificar automáticamente si los datos cumplen con esas condiciones.

Su utilidad principal está en detectar problemas de calidad antes de avanzar a etapas críticas como el modelado o la toma de decisiones. De esta manera, ayuda a prevenir errores costosos causados por datos incompletos, inconsistentes o fuera de rango.

## DATASET E-COMMERCE
El dataset de E-commerce utilizado en este trabajo corresponde a transacciones reales de una tienda en línea del Reino Unido, registradas entre los años 2010 y 2011. Contiene información detallada sobre las facturas emitidas, los productos vendidos, las cantidades adquiridas, los precios unitarios, así como datos de los clientes y su país de origen.

### 0) Cargar dataset (RAW)

In [1]:
# IMPORTACIONES
import numpy, scipy, pandas, sklearn, great_expectations as ge
print("NumPy:", numpy.__version__)
print("SciPy:", scipy.__version__)
print("Pandas:", pandas.__version__)
print("Sklearn:", sklearn.__version__)
print("GE:", ge.__version__)

NumPy: 1.26.4
SciPy: 1.10.1
Pandas: 2.2.2
Sklearn: 1.5.1
GE: 0.18.13


In [6]:
from pathlib import Path
import pandas as pd
import numpy as np

DATA_PATH = Path("../data/e-commerce.csv")
print("Usando:", DATA_PATH.resolve())

df_raw = pd.read_csv(DATA_PATH, encoding_errors="ignore")
print("RAW shape:", df_raw.shape)
df_raw.head(3)

Usando: /Users/daniel/Documents/cursos/maestria-ia/modulo-7/titanic/data/e-commerce.csv
RAW shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


#### Descripción de las columnas
- **InvoiceNo** Número de factura. Es un identificador único para cada transacción.

- **StockCode** Código único asignado a cada producto. Sirve para identificar el artículo en inventario.

- **Description**
Nombre o descripción textual del producto.

- **Quantity**
Número de unidades vendidas en cada transacción.

- **InvoiceDate**
Fecha y hora exacta en que se generó la factura.

- **UnitPrice**
Precio unitario del producto, expresado en libras esterlinas (£).

- **CustomerID**
Identificador numérico único de cada cliente.

- **Country**
País del cliente. Indica desde dónde se realizó la compra.

### 1) Preprocesamiento mínimo (tipos)
Objetivo: Asegurar que las variables principales tengan el tipo de dato correcto y que no haya problemas básicos al analizarlas.

In [7]:
df_raw = df_raw.copy()

# Fecha → datetime
if "InvoiceDate" in df_raw.columns:
    df_raw["InvoiceDate"] = pd.to_datetime(df_raw["InvoiceDate"], errors="coerce")

# Números → asegurar tipo
for col in ["Quantity", "UnitPrice", "CustomerID"]:
    if col in df_raw.columns:
        df_raw[col] = pd.to_numeric(df_raw[col], errors="coerce")

print(df_raw.dtypes)
df_raw.isna().sum().sort_values(ascending=False).head(10)


InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object


CustomerID     135080
Description      1454
InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

### 2) Great Expectations — contexto y datasource (idempotente)
Objetivo: Paso para que Great Expectations “reconozca” y se conecte a nuestro dataframe de e-commerce. Sin este contexto y datasource, no podríamos ejecutar reglas de calidad sobre los datos.

In [8]:
import great_expectations as ge
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.exceptions import DataContextError

# Contexto efímero en notebook
context = ge.get_context()

# Datasource runtime para Pandas (seguro si ya existe)
try:
    context.add_datasource(
        name="runtime_pandas_datasource",
        class_name="Datasource",
        execution_engine={"class_name": "PandasExecutionEngine"},
        data_connectors={
            "runtime_data_connector": {
                "class_name": "RuntimeDataConnector",
                "batch_identifiers": ["default_identifier_name"],
            }
        },
    )
except DataContextError:
    pass  # ya existe

### 3) VALIDACIÓN RAW (diagnóstico)
Objetivo: Chequeo médico inicial, confirma que el dataset es coherente en lo más básico, y que no hay problemas graves que impidan seguir adelante.

In [10]:
from great_expectations.core.expectation_suite import ExpectationSuite

# Batch request usando df_raw en memoria
batch_request_raw = RuntimeBatchRequest(
    datasource_name="runtime_pandas_datasource",
    data_connector_name="runtime_data_connector",
    data_asset_name="ecommerce_raw",
    runtime_parameters={"batch_data": df_raw},
    batch_identifiers={"default_identifier_name": "raw_run"},
)

# Crear/registrar suite RAW (0.18.x usa add_or_update)
suite_name_raw = "ecommerce_raw_suite"
try:
    suite_raw = context.get_expectation_suite(suite_name_raw)
except DataContextError:
    suite_raw = ExpectationSuite(expectation_suite_name=suite_name_raw)
    context.add_or_update_expectation_suite(expectation_suite=suite_raw)

# Obtener validator RAW
validator_raw = context.get_validator(
    batch_request=batch_request_raw,
    expectation_suite_name=suite_name_raw
)

# ---- EXPECTATIONS (menos estrictas, diagnóstico) ----
present = [c for c in ["InvoiceNo","StockCode","Description","Quantity",
                       "InvoiceDate","UnitPrice","CustomerID","Country"]
           if c in df_raw.columns]
validator_raw.expect_table_columns_to_match_set(present)

for c in ["InvoiceNo","StockCode","InvoiceDate","UnitPrice","Country"]:
    if c in df_raw.columns:
        validator_raw.expect_column_values_to_not_be_null(c, mostly=0.95)

if "UnitPrice" in df_raw.columns:
    validator_raw.expect_column_values_to_be_between("UnitPrice", min_value=0, max_value=None, mostly=0.99)
    #validator_raw.expect_column_values_to_be_greater_than_or_equal_to("UnitPrice", 0)

if "InvoiceDate" in df_raw.columns:
    validator_raw.expect_column_values_to_be_between(
        "InvoiceDate",
        min_value=pd.Timestamp("2010-12-01"),
        max_value=pd.Timestamp("2011-12-09"),
        parse_strings_as_datetimes=True,
        mostly=0.97
    )

if "InvoiceNo" in df_raw.columns:
    validator_raw.expect_column_values_to_match_regex("InvoiceNo", r"^[C]?\d+$", mostly=0.97)

if "Country" in df_raw.columns:
    top_countries = df_raw["Country"].dropna().value_counts().head(30).index.tolist()
    validator_raw.expect_column_values_to_be_in_set("Country", top_countries, mostly=0.98)

# Guardar suite y validar
validator_raw.save_expectation_suite(discard_failed_expectations=False)
res_raw = validator_raw.validate()
res_raw["statistics"]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/30 [00:00<?, ?it/s]

{'evaluated_expectations': 10,
 'successful_expectations': 10,
 'unsuccessful_expectations': 0,
 'success_percent': 100.0}

#### Qué muestran los resultados
- evaluated_expectations: 10 → Se evaluaron 10 reglas de calidad de datos (expectations).
- successful_expectations: 10 → Las 10 se cumplieron.
- unsuccessful_expectations: 0 → Ninguna falló.
- success_percent: 100.0 → Tasa de éxito del 100%.
Las barras verdes (Calculating Metrics: 100%) reflejan que todas las comprobaciones terminaron correctamente y dentro de lo esperado.

#### Interpretación
- **Consistencia básica garantizada**
El dataset cumple con las reglas mínimas que definimos: columnas presentes, valores no nulos en las principales, precios unitarios no negativos, fechas dentro del rango esperado, etc.

- **No se detectaron anomalías graves en RAW**
Esto significa que tus datos crudos ya están bastante “limpios” en términos estructurales.
Ojo: esto no garantiza que el dataset esté perfecto (puede haber outliers, duplicados, registros inconsistentes en negocio), pero sí que pasó las validaciones básicas que configuramos.

- **Base sólida para limpieza y modelado**
Tener 100% aquí es una buena señal: no es necesario hacer correcciones drásticas antes de pasar a la fase CLEAN o al modelado posterior.

### 4) Limpieza mínima
Este paso representó un filtro inicial para quedarnos con datos más confiables:
- Quitamos errores evidentes (cantidades o precios negativos).
- Generamos una métrica clave (Revenue).
- Comprobamos que el dataset mejoró, aunque una de las expectativas falló (lo que revela la presencia de casos atípicos que aún requieren análisis).

En resumen: el dataset pasara de ser “utilizable” a ser “más limpio y coherente”, aunque todavía no está perfecto.

In [11]:
df_clean = df_raw.copy()

# UnitPrice no negativo (devoluciones se reflejan en Quantity)
if "UnitPrice" in df_clean.columns:
    df_clean = df_clean[df_clean["UnitPrice"].fillna(0) >= 0]

# Quantity entero (si vino como float por nulos)
if "Quantity" in df_clean.columns and not np.issubdtype(df_clean["Quantity"].dtype, np.integer):
    df_clean["Quantity"] = pd.to_numeric(df_clean["Quantity"], errors="coerce").fillna(0).astype(int)

# Derivar Revenue
if set(["Quantity","UnitPrice"]).issubset(df_clean.columns):
    df_clean["Revenue"] = df_clean["Quantity"] * df_clean["UnitPrice"]

# Quitar duplicados exactos
df_clean = df_clean.drop_duplicates()

print("CLEAN shape:", df_clean.shape)
df_clean.head(3)

CLEAN shape: (536639, 9)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00


### 5) VALIDACIÓN CLEAN (más estricta)

In [13]:
# Batch request usando df_clean en memoria
batch_request_clean = RuntimeBatchRequest(
    datasource_name="runtime_pandas_datasource",
    data_connector_name="runtime_data_connector",
    data_asset_name="ecommerce_clean",
    runtime_parameters={"batch_data": df_clean},
    batch_identifiers={"default_identifier_name": "clean_run"},
)

# Crear/registrar suite CLEAN
suite_name_clean = "ecommerce_clean_suite"
try:
    suite_clean = context.get_expectation_suite(suite_name_clean)
except DataContextError:
    suite_clean = ExpectationSuite(expectation_suite_name=suite_name_clean)
    context.add_or_update_expectation_suite(expectation_suite=suite_clean)

# Validator CLEAN
validator_clean = context.get_validator(
    batch_request=batch_request_clean,
    expectation_suite_name=suite_name_clean
)

# ---- EXPECTATIONS (más estrictas) ----
expected_clean = [c for c in ["InvoiceNo","StockCode","Description","Quantity",
                              "InvoiceDate","UnitPrice","CustomerID","Country","Revenue"]
                  if c in df_clean.columns]
validator_clean.expect_table_columns_to_match_set(expected_clean)

for c in ["InvoiceNo","StockCode","InvoiceDate","UnitPrice","Country"]:
    if c in df_clean.columns:
        validator_clean.expect_column_values_to_not_be_null(c)

if "UnitPrice" in df_clean.columns:
    validator_clean.expect_column_values_to_be_between("UnitPrice", min_value=0, max_value=None)
    #validator_clean.expect_column_values_to_be_greater_than_or_equal_to("UnitPrice", 0)

if "Revenue" in df_clean.columns and df_clean["Revenue"].notna().sum() > 0:
    q01, q99 = df_clean["Revenue"].quantile([0.01, 0.99])
    validator_clean.expect_column_values_to_be_between("Revenue", min_value=q01, max_value=q99, mostly=0.98)

if "InvoiceNo" in df_clean.columns:
    validator_clean.expect_column_values_to_match_regex("InvoiceNo", r"^[C]?\\d+$")

validator_clean.save_expectation_suite(discard_failed_expectations=False)
res_clean = validator_clean.validate()
res_clean["statistics"]

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/29 [00:00<?, ?it/s]

{'evaluated_expectations': 9,
 'successful_expectations': 8,
 'unsuccessful_expectations': 1,
 'success_percent': 88.88888888888889}

#### Qué muestran los resultados
- evaluated_expectations: 9 → Se aplicaron 9 reglas de calidad sobre los datos ya limpiados.
- successful_expectations: 8 → Ocho se cumplieron.
- unsuccessful_expectations: 1 → Una falló.
- success_percent: 88.9% → La tasa de éxito global fue del 89%.

### Interpretación

- **Los datos mejoraron respecto al RAW**
Ahora estás aplicando reglas más estrictas, como la validez del campo Revenue o límites basados en cuantiles. Que la mayoría se cumpla (89%) indica que tu limpieza va por buen camino.

- **Un fallo significa una alerta, no necesariamente un error grave**
La expectativa que falló probablemente esté ligada a algún dato que se sale de los rangos esperados (ej. valores de Revenue fuera de los cuantiles definidos, o algún InvoiceNo con formato inusual). Esto no invalida el dataset, pero sí señala que hay outliers o casos atípicos que vale la pena revisar.

- **El dataset CLEAN es más confiable que el RAW**
El porcentaje de éxito sigue siendo alto (casi 9/10 reglas pasaron). Esto muestra que la mayoría de los datos cumplen con criterios de negocio básicos, y que las inconsistencias son puntuales.